#데이터 증가
## 기존의 데이터를 변형해서 데이터의 양을 늘리는 것을 데이터 증강
- Keras의 ImageDataGenerator를 사용해서 데이터 증강을 확인

#1. 데이터 준비
- 이미지가 저장된 폴더 설정

In [ ]:
train_dir = './datasets/dogs-vs-cats/train'
val_dir = './datasets/dogs-vs-cats/val'
test_dir = './datasets/dogs-vs-cats/test'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='reflect')
val_datagen = ImageDataGenerator(rescale=1./ 255)
test_datagen = ImageDataGenerator(rescale=1./ 255)

#2. 데이터셋 객체 생성
- ImageDataGenerator 객체에 데이터 폴더를 설정
- 개와 고양이 이진 분류이므로 class_
mode는 ‘binary’로 설정

In [ ]:
MAGE_SIZE = (224, 224)
BATCH_SIZE = 128
train_ds = train_datagen.flow_from_directory(train_dir,
                       batch_size=BATCH_SIZE,
                       target_size=IMAGE_SIZE,
                       class_mode='binary')
valid_ds = val_datagen.flow_from_directory(val_dir,
                       batch_size=BATCH_SIZE,
                       target_size=IMAGE_SIZE,
                       class_mode='binary')
test_ds = test_datagen.flow_from_directory(test_dir,
                       batch_size=BATCH_SIZE,
                       target_size=IMAGE_SIZE,
                       class_mode='binary')

#3. 모델 만들기

- 출력층의 출력 개수와 활성화 함수에 주의
- 이번 모델은 이진 분류

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
def build_model():
   model = keras.Sequential([
     layers.Conv2D(64, (3, 3), activation='relu',
                   input_shape=(224, 224, 3)),
     layers.MaxPooling2D(2, 2),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D(2, 2),
     layers.Conv2D(128, (3, 3), activation='relu'),
     layers.MaxPooling2D(2, 2),
     layers.Flatten(),
     layers.Dropout(0.5),
     layers.Dense(512, activation='relu'),
     layers.Dense(1, activation='sigmoid'),
   ])
   return model

#4. 콜백 생성

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "./temp/dogs-vs-cats.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                save_weights_only=True,
                save_best_only=True,
                monitor='val_loss',
                verbose=1)

#5. 학습

In [ ]:
model = build_model()
model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['acc'])
EPOCHS = 25
history = model.fit(train_ds,
           validation_data=valid_ds,
           epochs=EPOCHS,
           callbacks=[checkpoint])

#6. 학습곡선

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  plt.figure(figsize=(16,8))
  plt.subplot(1,2,1)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'], hist['loss'], label='Train Loss')
  plt.plot(hist['epoch'], hist['val_loss'],label = 'Val Loss')
  plt.legend()

  plt.subplot(1,2,2)
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(hist['epoch'], hist['acc'], label='Train Accuracy')
  plt.plot(hist['epoch'], hist['val_acc'], label = 'Val Accuracy')
  plt.legend()
plt.show()

In [ ]:
plot_history(history)

#7. 가중치 로딩

In [ ]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_ds)
print(loss, acc)

#8. 모델 저장
### 학습 완료된 모델을 저장
- 저장 방법은 2가지
- h5 확장자로 하나의 파일로 저장

In [ ]:
model.save('dogs-vs-cats.h5')

In [ ]:
model.save('dogs-vs-cats')

#9. 모델 로딩
- 파일로 저장된 모델을 로딩하는 방법

In [ ]:
from tensorflow.keras import models
# mymodel = models.load_model('dogs-vs-cats.h5')
mymodel = models.load_model('mymodel')

#10. 모델 확인
- 로딩된 모델을 평가


In [ ]:
loss, acc = mymodel.evaluate(test_ds)

---------------------------

# *** 실습 ***